# Varying λ Values
## Dataset: Lund2013

In [ ]:
import time

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

import Config.constants as cnst
from GazeDetectors.EngbertDetector import EngbertDetector
from DataSetLoaders.DataSetFactory import DataSetFactory

import Analysis.comparisons as cmps
import Analysis.scarfplot as scarfplot
import Analysis.figures as figs

pio.renderers.default = "browser"

In [ ]:
LAMBDA = "λ"

DATASET_NAME = "Lund2013"
RATERS = ["MN", "RA"]
DETECTORS = [EngbertDetector(lambdaa=lmda) for lmda in np.arange(1, 7, 1)]

rename_columns = lambda col: col[col.index(LAMBDA):col.index(",")].replace("'", "") if LAMBDA in col else col
COMPARISON_COLUMNS = [(RATERS[1], rename_columns(d.name)) for d in DETECTORS]
EVENT_MATCHING_PARAMS = {"match_by": "onset", "max_onset_latency": 15, "allow_cross_matching": False}

## Load & Process

In [ ]:
start = time.time()

lund_dataset = DataSetFactory.load(DATASET_NAME)
lund_samples, lund_events, lund_detector_res = DataSetFactory.process(lund_dataset, RATERS, DETECTORS)

lund_samples.rename(columns=rename_columns, inplace=True)
lund_events.rename(columns=rename_columns, inplace=True)
lund_detector_res.rename(columns=rename_columns, inplace=True)

end = time.time()
print(f"Finished Detecting Events:\t{end - start:.2f} seconds")
del start, end

## Detection Scarfplot

In [ ]:
SCARF_WIDTH = 10
STIM = "img"  # "video" / "moving dot"

for i, idx in enumerate(lund_samples.index):
    if idx[2] != STIM:
        continue
    t = np.arange(len(lund_samples.iloc[i]["RA"]))  # use RA as ground-truth for number of samples
    scarfplot_fig = go.Figure()
    for j, col in enumerate(lund_samples.columns):
        labels = lund_samples.loc[idx, col]
        if pd.isna(labels).all():
            # ignore undetected trials
            continue
        scarfplot_fig = scarfplot.add_scarfplot(scarfplot_fig, t, labels,
                                                ymin=2 * j * SCARF_WIDTH,
                                                ymax=(2 * j + 1) * SCARF_WIDTH)
    scarfplot_fig.update_layout(title=f"Subject: {idx[1]}\t\tStimulus: {idx[2]}\t\tStimulus Name: {idx[3]}",
                                xaxis=dict(title="Samples", showgrid=False),
                                yaxis=dict(range=[0, (2 * len(lund_samples.columns) - 1) * SCARF_WIDTH],
                                           tickmode='array',
                                           tickvals=[(2 * i + 0.5) * SCARF_WIDTH for i in
                                                     range(len(lund_samples.columns))],
                                           ticktext=list(lund_samples.columns))
                                )
    scarfplot_fig.show()

del i, j, idx, col, t, labels

## Compare against GT
Compare the detected outputs against the human-annotated GT, with metrics:  
(1) Label Counts  
(2) Cohen's Kappa  
(3) Matched-Events Ratio  

In [ ]:
label_counts = cmps.label_counts(lund_samples, group_by=cnst.STIMULUS)
label_counts_fig = figs.count_grid(label_counts,
                                   title="Label Counts")
label_counts_fig.show()

In [ ]:
cohen_kappa = cmps.compare_samples(lund_samples, metric='kappa', group_by=cnst.STIMULUS)
cohen_kappa_fig = figs.distributions_grid(cohen_kappa[COMPARISON_COLUMNS],
                                          plot_type="violin",
                                          title="Cohen's Kappa",
                                          column_title_mapper=lambda col: f"{col[0]}→{col[1]}")
cohen_kappa_fig.show()

In [ ]:
matching_ratio = cmps.event_matching_ratio(lund_events, group_by=cnst.STIMULUS, **EVENT_MATCHING_PARAMS)
matching_ratio_fig = figs.distributions_grid(matching_ratio[COMPARISON_COLUMNS],
                                             plot_type="violin",
                                             title="Event Matching Ratio",
                                             column_title_mapper=lambda col: f"{col[0]}→{col[1]}")
matching_ratio_fig.show()

## Velocity Threshold Distribution

In [ ]:
thresholds = pd.concat([lund_detector_res[f"{LAMBDA}:1"].map(lambda cell: cell['thresh_Vx']),
                        lund_detector_res[f"{LAMBDA}:1"].map(lambda cell: cell['thresh_Vy'])],
                       axis=1, keys=["Vx", "Vy"])
agg_thresholds = cmps.group_and_aggregate(thresholds, group_by=cnst.STIMULUS)
threshold_distribution_fig = figs.distributions_grid(agg_thresholds,
                                                     plot_type="violin",
                                                     title="Thresholds Distribution")
threshold_distribution_fig.show()